# Tango Solver (LinkedIn) - by Darci Peoples

### Puzzle Data Structure

In [13]:
# This cell has the puzzle, solver, and text input parser
# TODO: Split up code into multiple classes and files

import math
import time

from collections import Counter, defaultdict
from enum import Enum
from typing import List, Dict, Tuple
from abc import abstractmethod, ABC
from copy import deepcopy

# Format a string with ANSI color for pretty text output
def fmt(string: str, code: int, bg=False, reset=True):
  return u"\u001b[" + f"{48 if bg else 38};5;{code}m{string}" + ("\u001b[0m" if reset else "")

# Enum of cell symbols
# LinkedIn uses moon/sun, Tango app uses circle/diamond, program text input uses B/Y
class Cell(Enum):
  BLUE = 'B'
  YELLOW = 'Y'
  BLANK = ' '

# Cell formatting for puzzle printing
CELL_FMT = {Cell.BLANK: fmt("   ", 8, True), Cell.BLUE: fmt(" ☾ ", 12, True), Cell.YELLOW: fmt(" ☼ ", 11, True)}

# Enum of edge types
# LinkedIn uses =/x, Tango app uses =/lightning, program text input uses =/x
class Edge(Enum):
  SAME = '='
  DIFFERENT = 'x'
  BLANK = ' '

# Edge formatting for puzzle printing
EDGE_TO_STR = {Edge.BLANK: u"   ", Edge.SAME: f' {fmt("=", 10)} ', Edge.DIFFERENT: f' {fmt("x", 9)} '}

# Type alias for a list of 2D indices
Indices = List[Tuple[int, int]]

# Map to swap between blue and yellow symbols
OPPOSITE_SYMBOL = {Cell.BLUE: Cell.YELLOW, Cell.YELLOW: Cell.BLUE}

# Puzzle class, for storing the cells and edges, parsing from text input, and printing
# TODO: Move parsing from text input to different class. Do the same for image inputs.
class Puzzle:
  cells: List[List[Cell]]
  edges: Dict[Tuple[int, int], Dict[Tuple[int, int], Edge]]

  ROWS: int
  COLS: int

  # Initialize a puzzle from an input file or from cells and edges
  def __init__(self, cells, edges):
    self.cells = cells
    self.edges = edges
    self.ROWS = len(cells)
    self.COLS = len(cells[0])
  
  # Check if two puzzles are the same
  def __eq__(self, other):
    return self.ROWS == other.ROWS and \
      self.COLS == other.COLS and \
        self.cells == other.cells and \
          self.edges == other.edges

  # Print the puzzle with pretty formatting
  def print(self):
    output = [['   ' for _ in range(self.COLS * 2 + 1)] for _ in range(self.ROWS * 2 + 1)]

    # Add cells to the output
    for i in range(self.ROWS):
      for j in range(self.COLS):
        output_i, output_j = i * 2 + 1, j * 2 + 1
        output[output_i][output_j] = CELL_FMT[self.cells[i][j]]
    
    # Add edges to the output
    for cell1 in self.edges:
      for cell2 in self.edges[cell1]:
        edge = self.edges[cell1][cell2]
        r1, c1 = cell1
        r2, c2 = cell2
        if r1 < r2: # top, bottom
          output_i, output_j = r1 * 2 + 2, c1 * 2 + 1
          output[output_i][output_j] = EDGE_TO_STR[edge]
        if c1 < c2: # left, right
          output_i, output_j = r1 * 2 + 1, c1 * 2 + 2
          output[output_i][output_j] = EDGE_TO_STR[edge]

    # Format a row or column count
    def fmt_count(count: int, N: int):
      return fmt(count, 8 if count == N // 2 else 9 if count > N // 2 else 255)
    
    # Calculate the row and column counts
    row_counts = [self.count_row(r) for r in range(self.ROWS)]
    col_counts = [self.count_col(c) for c in range(self.COLS)]
    
    # Add row counts to output
    for i, line in enumerate(output):
      if i % 2 == 1:
        R = self.ROWS
        blue, yellow = row_counts[i//2][Cell.BLUE], row_counts[i//2][Cell.YELLOW]
        count_str = f"{fmt_count(blue, R)}{fmt('|', 8)}{fmt_count(yellow, R)}"
        output[i] = [count_str] + line + [count_str]
      else:
        output[i] = ["   "] + line + ["   "]

    # Add col counts to output
    col_count_strs = ['   ']
    for c in range(self.COLS):
      C = self.COLS
      blue, yellow = col_counts[c][Cell.BLUE], col_counts[c][Cell.YELLOW]
      count_str = f"{fmt_count(blue, C)}{fmt('|', 8)}{fmt_count(yellow, C)}"
      col_count_strs.append(count_str)
    output.insert(0, ['   '.join(col_count_strs)])
    output.append(['   '.join(col_count_strs)])
    
    # Print the output
    print('\n' + '\n'.join([''.join(line) for line in output]) + '\n')
  
  # Count the frequency of each symbol in this line
  def count_line(self, indices: Indices):
    cells = self.cells
    counts = Counter()
    for r, c in indices:
      marker = cells[r][c]
      counts[marker] += 1
    return counts

  # Count the frequency of each symbol in this row
  def count_row(self, r: int):
    return self.count_line([(r, c) for c in range(self.COLS)])
  
  # Count the frequency of each symbol in this column
  def count_col(self, c: int):
    return self.count_line([(r, c) for r in range(self.ROWS)])

### Text Parsing

In [14]:
# Parse a puzzle from text input (see example .txt files)
def parse_text_file(input_file):
  # Read the input file
  puzzle_input = [list(x.strip()) for x in open(input_file).readlines()]
  
  # Calc the puzzle dimensions
  R = len(puzzle_input) // 2
  C = len(puzzle_input[0]) // 2

  # Parse cells and edges from input text
  cells = [[Cell.BLANK for _ in range(C)] for _ in range(R)]
  edges = defaultdict(dict)
  for i in range(1, len(puzzle_input) - 1):
    for j in range(1, len(puzzle_input[i]) - 1):
      char = puzzle_input[i][j]
      # Parse cells
      if j % 2 == 1 and i % 2 == 1:
        cells[i // 2][j // 2] = Cell(char)

      # Parse edges that connect cells vertically
      if i % 2 == 0 and j % 2 == 1:
          top = (i - 1) // 2, j // 2
          bottom = (i + 1) // 2, j // 2
          if char in (Edge.DIFFERENT.value, Edge.SAME.value):
            edges[top][bottom] = edges[bottom][top] = Edge(char)

      # Parse edges that connect cells horizontally
      if i % 2 == 1 and j % 2 == 0:
        left = i // 2, (j - 1) // 2
        right = i // 2, (j + 1) // 2
        if char in (Edge.DIFFERENT.value, Edge.SAME.value):
          edges[left][right] = edges[right][left] = Edge(char)

  # Create a puzzle using these cells and edges
  edges = dict(edges)
  return Puzzle(cells, edges)

### Solver

In [15]:
# An abstract class for making rules that operate on each line (row or col) independently
# TODO: Dedupe with LinewiseStrat
class LinewiseRule(ABC):
  # Check this line
  @staticmethod
  @abstractmethod
  def check_line(puzz, indices: Indices):
    raise Exception("LinewiseRule check not implemented")
  
  # Check this column
  @classmethod
  def check_col(cls, puzz, c: int):
    return cls.check_line(puzz, [(r, c) for r in range(puzz.ROWS)])
  
  # Check this row
  @classmethod
  def check_row(cls, puzz, r: int):
    return cls.check_line(puzz, [(r, c) for c in range(puzz.COLS)])
  
  # Check all rows and columns
  @classmethod
  def check(cls, puzz):
    return all([cls.check_row(puzz, r) for r in range(puzz.ROWS)]) and \
      all([cls.check_col(puzz, c) for c in range(puzz.COLS)])

# Check that every line has an equal number of blue and yellow
class SameNumberRule(LinewiseRule):
  # Check that no symbol takes up more than half the cells
  @staticmethod
  def check_line(puzz, indices: Indices):
    N = len(indices)
    counts = puzz.count_line(indices)
    return counts[Cell.BLUE] <= N // 2 and counts[Cell.YELLOW] <= N // 2

  # TODO: Add a done applying function to each rule/strat so we can stop checking?
  def done_applying(puzz):
    pass

# Check that there is <=2 symbols of the same kind next to each other
class TwoInARowRule(LinewiseRule):
  # Check that the line doesn't have any runs of length >2
  @staticmethod
  def check_line(puzz, indices: Indices):
    cells = puzz.cells
    # Track the current symbol and run length 
    curr_symbol = None
    curr_run_len = 0
    # Iterate through the line, looking for runs >2 in length
    for (r, c) in indices:
      cell = cells[r][c]
      # If we hit a blank, reset the run length
      if cell == Cell.BLANK:
        curr_run_len = 0
        continue
      # If the symbol changed, reset the run length
      if cell != curr_symbol:
        curr_symbol = cell
        curr_run_len = 0
      # Increment the run length
      curr_run_len += 1
      # If the run length >2, fail the check
      if curr_run_len > 2:
        return False
    return True

# Check that the '=' and 'x' edge conditions are followed
class EdgesRule:
  @staticmethod
  def check(puzz):
    # Check that every edge condition holds
    for cell1 in puzz.edges:
      for cell2 in puzz.edges[cell1]:
        # Skip re-checking edges in the opposite direction
        if cell1 > cell2:
          continue
        edge = puzz.edges[cell1][cell2]
        r1, c1 = cell1
        r2, c2 = cell2
        symbol1 = puzz.cells[r1][c1]
        symbol2 = puzz.cells[r2][c2]
        # If either side is blank, no violate
        if symbol1 == Cell.BLANK or symbol2 == Cell.BLANK:
          continue
        # If a SAME edge, but not equal, fail
        if edge == Edge.SAME:
          if symbol1 != symbol2:
            return False
        # If a DIFFERENT edge, but equal, fail
        if edge == Edge.DIFFERENT:
          if symbol1 == symbol2:
            return False
    # Pass if we didn't find any edge violations
    return True

# An abstract linewise strategy for making classes that operate on each line independently
class LinewiseStrat:
  # Apply to this line
  @staticmethod
  @abstractmethod
  def apply_to_line(puzz, indices: Indices):
    raise Exception("LinewiseStrat apply not implemented")
  
  # Apply to this column
  @classmethod
  def apply_to_col(cls, puzz, c: int):
    cls.apply_to_line(puzz, [(r, c) for r in range(puzz.ROWS)])
  
  # Apply to this row
  @classmethod
  def apply_to_row(cls, puzz, r: int):
    cls.apply_to_line(puzz, [(r, c) for c in range(puzz.COLS)])
  
  # Apply to every row and column
  @classmethod
  def apply(cls, puzz):
    for r in range(puzz.ROWS):
      cls.apply_to_row(puzz, r)
    for c in range(puzz.COLS):
      cls.apply_to_col(puzz, c)

# TODO: Merge rules and strategies?
# Cap each pair with the opposite symbol
# E.g. _BB_ -> YBBY
class CapPairsStrat(LinewiseStrat):
  # Cap pairs on the given line 
  @staticmethod 
  def apply_to_line(puzz, indices: Indices):
    # Only need to apply if line has 3+ elements
    N = len(indices)
    if N <= 2:
      return

    # Figure out if this is a row or col and its index, for error messages
    line_desc = "column" if indices[0][0] == indices[1][0] else "row"
    line_num = indices[0][0] if indices[0][0] == indices[1][0] else indices[0][1]
      
    cells = puzz.cells
    # Track the current symbol and run length 
    curr_symbol = None
    curr_run_len = 0

    # Iterate through the line, looking for pairs
    for i, (r, c) in enumerate(indices):
      cell = cells[r][c]
      # If we see a blank, reset the run length
      if cell == Cell.BLANK:
        curr_run_len = 0
        continue
      # If the symbol changed, reset the run length
      if curr_symbol != cell:
        curr_run_len = 0
        curr_symbol = cell
      # Increment run length
      curr_run_len += 1
      # Double-check that there's no run of three
      assert curr_run_len <= 2
      # If we found a run of two, cap both sides with the opposite
      if curr_run_len == 2:
        # Set symbol 2 indices ago to opposite
        if i >= 2:
          tr, tc = indices[i-2]
          if cells[tr][tc] == Cell.BLANK:
            cells[tr][tc] = OPPOSITE_SYMBOL[cell]
          else:
            # TODO: Throw a contradiction error, we should not keep trying
            pass
        # Set symbol 1 index later to opposite
        if i < N - 1:
          tr, tc = indices[i+1]
          if cells[tr][tc] == Cell.BLANK:
            cells[tr][tc] = OPPOSITE_SYMBOL[cell]
          else:
            # TODO: Throw a contradiction error, we should not keep trying
            pass

# TODO: Merge with two in a row rule
# If there are two of the same symbol separated by a blank, fill with the opposite
# E.g. B_B -> BYB
class FillGapStrat(LinewiseStrat):
  @staticmethod 
  def apply_to_line(puzz, indices: Indices):
    N = len(indices)
    cells = puzz.cells
    # Iterate through the line, filling pair gaps if we find them
    for i in range(1, N - 1):
      r, c = indices[i]
      cell = cells[r][c]
      pr, pc = indices[i-1]
      prev_cell = cells[pr][pc]
      nr, nc = indices[i+1]
      next_cell = cells[nr][nc]
      # If we found a blank between two symbols of the same kind, fill the gap with the opposite
      if cell == Cell.BLANK and prev_cell == next_cell and prev_cell != Cell.BLANK:
        cells[r][c] = OPPOSITE_SYMBOL[prev_cell]

# Populate cells using edge conditions, as long as we know one side
class EdgesStrat:
  # Populate the other side of every edge condition
  @staticmethod 
  def apply(puzz):
    cells = puzz.cells
    # Iterate through each edge, looking for cells we can populate
    for cell1 in puzz.edges:
      for cell2 in puzz.edges[cell1]:
        # Skip re-applying to edges in the opposite direction
        # TODO: Remove cell2 -> cell1 edges, we don't need both really
        if cell1 > cell2:
          continue
        edge = puzz.edges[cell1][cell2]
        r1, c1 = cell1
        r2, c2 = cell2
        marker1 = cells[r1][c1]
        marker2 = cells[r2][c2]
        # If cell 1 is blank and cell 2 isn't, fill cell 1
        if marker1 == Cell.BLANK and marker2 != Cell.BLANK:
          # _ = A -> A = A
          if edge == Edge.SAME:
            cells[r1][c1] = marker2
          # _ x B -> A x B
          elif edge == Edge.DIFFERENT:
            cells[r1][c1] = OPPOSITE_SYMBOL[marker2]
        # If cell 2 is blank and cell 1 isn't, fill cell 2
        elif marker2 == Cell.BLANK and marker1 != Cell.BLANK:
          # A = _ -> A = A
          if edge == Edge.SAME:
            cells[r2][c2] = marker1
          # A x _ -> A x B
          elif edge == Edge.DIFFERENT:
            cells[r2][c2] = OPPOSITE_SYMBOL[marker1]

# TODO: Merge with same number rule?
# Fill the rest of the line, if one symbol has hit its limit
class SameNumberStrat(LinewiseStrat):
  # If half the line is filled with a symbol, fill the blanks with the opposite
  @staticmethod
  def apply_to_line(puzz, indices: Indices):
    cells = puzz.cells
    N = len(indices)
    counts = puzz.count_line(indices)
    # For each symbol, check if we've hit the limit
    for marker in (Cell.BLUE, Cell.YELLOW):
      # If we have enough of one symbol and not enough of the other
      if counts[marker] == N // 2 and counts[OPPOSITE_SYMBOL[marker]] < N // 2:
        # Fill the rest of the blanks with the opposite symbol
        for r, c in indices:
          cell = cells[r][c]
          if cell == Cell.BLANK:
            cells[r][c] = OPPOSITE_SYMBOL[marker]
        # We don't need to check the other symbol
        return

# TODO: Apply once?
# TODO: Remove edge conditions if they're done being used
# If there's an = edge condition, cap the cells with x edge conditions
# E.g. _ _=_ _ -> _x_=_x_
class CapEqualEdgeStrat:
  @staticmethod
  def apply(puzz):
    R, C = puzz.ROWS, puzz.COLS
    # Track which edges we want to add, since we can't make changes in the for loop
    new_edges = defaultdict(dict)
    # Iterate through the edges
    for cell1 in puzz.edges:
      for cell2 in puzz.edges[cell1]:
        edge = puzz.edges[cell1][cell2]
        # Skip re-checking edges in the opposite direction
        # And only check '=' edges
        if edge != Edge.SAME or cell1 > cell2:
          continue
        r1, c1 = cell1
        r2, c2 = cell2
        # Figure out if this is a row
        is_row = r1 == r2
        # Find the cells on both ends of the '=' pair. Deduce whether left/right or top/bottom based on whether it's a row
        prev_cell = (r1, c1 - 1) if is_row and c1 > 0 else (r1 - 1, c1) if not is_row and r1 > 0 else None
        next_cell = (r2, c2 + 1) if is_row and c2 < C - 1 else (r2 + 1, c2) if not is_row and r2 < R - 1 else None
        # Cap the previous cell
        if prev_cell is not None:
          new_edges[prev_cell][cell1] = Edge.DIFFERENT
        # Cap the next cell
        if next_cell is not None:
          new_edges[cell2][next_cell] = Edge.DIFFERENT

    # Add these new edges to the puzzle (both directions)
    # Iterate through the new edges
    for cell1 in new_edges:
      for cell2 in new_edges[cell1]:
        edge = new_edges[cell1][cell2]
        if cell1 not in puzz.edges:
          puzz.edges[cell1] = {}
        if cell2 not in puzz.edges:
          puzz.edges[cell2] = {}
        # Double-check that we aren't overwriting an edge
        assert (cell2 not in puzz.edges[cell1] or puzz.edges[cell1][cell2] != Edge.SAME) and \
          (cell1 not in puzz.edges[cell2] or puzz.edges[cell2][cell1] != Edge.SAME)
        # Add both edge directions to the puzzle
        puzz.edges[cell1][cell2] = puzz.edges[cell2][cell1] = edge

# TODO: Forced triple strat will finish expert-10
# See if setting a given symbol will cause a triple later
# E.g. B ... B _ _ Y ... _ (n-1|n-2) -> B ... B _ _ Y ... Y (n-1|n-1) since B ... B _ _ Y ... B (n|n-2) causes invalid B ... B Y Y Y ... B (n|n).
class ForcedTriplesStrat:
  pass

# TODO: Make a strat to insert 'x' edges where that must be the case
# E.g. A _ _ B -> A _x_ B. This is helpful if we start using 'x' conditions to add to symbol counts
# E.g. _ _ _x_ (0|0) -> _x_ _x_ (0|0) maybe

# TODO: Make a strat to remove complete edges (for performance)
# TODO: Modify how edges are stored to keep track of which are original, which we added, and which are currently satisfied
# E.g. A=A -> A A, AxB -> A B

# TODO: Modify how cells are stored to keep track of which are original and maybe a list of blank cell indices, for performance

# TODO: Make a strat that includes 'x' edge conditions in line counts, to improve genius-1
# E.g. _ x _ ... _ (n-2, n-1) -> _ x _ ... B (n-1|n-1)
class IncludeDifferentEdgesInLineCountStrat:
  pass

# Apply each strategy to the puzzle (once)
def apply_strats(puzzle):
  CapPairsStrat.apply(puzzle)
  FillGapStrat.apply(puzzle)
  EdgesStrat.apply(puzzle)
  SameNumberStrat.apply(puzzle)
  CapEqualEdgeStrat.apply(puzzle)

# A puzzle is valid if all of the game rules hold
def is_valid(puzzle):
  return all([
    SameNumberRule.check(puzzle),
    TwoInARowRule.check(puzzle),
    EdgesRule.check(puzzle)
  ])

# Throw this if the puzzle is invalid, so we can backtrack
# TODO: Do we need to throw? When should we throw?
class HitContraditionError(Exception):
  pass

# Apply our rules over and over, until the puzzle stops changing
# In that case, we'll need guess.
# If the puzzle becomes invalid at any point, we'll need to backtrack
def deduce_all(puzzle):
  prev_puzzle = None
  # Keep going until the puzzle stops changing
  while not prev_puzzle or puzzle != prev_puzzle:
    prev_puzzle = deepcopy(puzzle)
    # Apply each of our strategies (once)
    apply_strats(puzzle)
    # Check if the puzzle became invalid (if so, we guessed wrong)
    # TODO: Return none instead?
    if not is_valid(puzzle):
      raise HitContraditionError("Puzzle became invalid during deduction")

# A puzzle is solved if it's complete (no blanks) and valid (no rule violations)
def is_solved(puzzle: Puzzle):
  return is_complete(puzzle) and is_valid(puzzle)

# A puzzle is complete if it has no blanks
# TODO: Make a helper in Puzzle that finds the total symbol counts
def is_complete(puzz: Puzzle):
  for r in range(puzz.ROWS):
    counter = puzz.count_row(r)
    if counter[Cell.BLANK] != 0:
      return False
  return True

# Solve the puzzle, if that's possible, using recursive backtracking
# TODO: Can we remove any of these is_valid checks?
# TODO: Can we remove contradiction error?
def solve(puzzle):
  # If the puzzle is invalid, there is no solution
  if not is_valid(puzzle):
    return None
  
  # Deduce whatever we can for the current puzzle
  try:
    deduce_all(puzzle)
  # If deduction hit a contradiction, there's no solution
  except HitContraditionError:
    return None

  # If the puzzle after deduction is invalid, there is no solution
  if not is_valid(puzzle):
    return None
  
  # If the puzzle is solved (valid and complete), we found the solution!
  if is_complete(puzzle):
    return puzzle
  
  # TODO: Make puzzle keep a list of blank cells maybe
  # Find the first blank cell
  first_blank = None
  for r in range(puzzle.ROWS):
    for c in range(puzzle.COLS):
      if puzzle.cells[r][c] == Cell.BLANK:
        first_blank = r, c
        break
    if first_blank is not None:
      break
  br, bc = first_blank
  
  # Guess that the first blank cell is BLUE
  puzzle.cells[br][bc] = Cell.BLUE
  # Try solving with this assumption
  soln = solve(deepcopy(puzzle))
  # Blue is correct! We found a solution
  if soln is not None:
    return soln

  # Guess that the first blank cell is YELLOW
  puzzle.cells[br][bc] = Cell.YELLOW
  # Try solving with this assumption
  soln = solve(deepcopy(puzzle))
  # Yellow is correct! We found a solution
  if soln is not None:
    return soln

  # Neither BLUE nor YELLOW worked out. The puzzle is unsolvable.
  return None

### Image Parsing

In [16]:
# This cell has generic image parsing helpers

from PIL import Image, ImageDraw
import numpy as np
import cv2
import statistics

# Check if inner box is contained in the outer box
def is_contained(inner, outer):
    x1, y1, x2, y2 = inner
    ox1, oy1, ox2, oy2 = outer
    return x1 >= ox1 and y1 >= oy1 and x2 <= ox2 and y2 <= oy2

# Check if two boxes overlap
def are_overlapping(rect_a, rect_b):
    ax1, ay1, ax2, ay2 = rect_a
    bx1, by1, bx2, by2 = rect_b
    return (ax1 <= bx1 <= ax2 and ay1 <= by1 <= ay2) or (ax1 <= bx2 <= ax2 and ay1 <= by2 <= ay2)

# Remove boxes that are nested within each other
def remove_nested_boxes(boxes):
    filtered = []
    for i, box in enumerate(boxes):
        if not any(i != j and is_contained(box, other) for j, other in enumerate(boxes)):
            filtered.append(box)
    return filtered

# Merge together overlapping boxes
# TODO: Not very efficient
def merge_overlapping_boxes(boxes):
    while True:
        new_boxes = set()
        overlapping_boxes = set()
        # Iterate over first boxes
        for i, box_a in enumerate(boxes):
            # Iterate over second boxes
            for j in range(i + 1, len(boxes)):
                box_b = boxes[j]
                # Check if the boxes overlap
                if are_overlapping(box_a, box_b):
                    overlapping_boxes.add(box_a)
                    overlapping_boxes.add(box_b)
                    # Add the merged box to our new list
                    new_boxes.add((min(box_a[0], box_b[0]), min(box_a[1], box_b[1]), max(box_a[2], box_b[2]), max(box_a[3], box_b[3])))
                    break
            # Add box A to our new list, if it wasn't merged
            if box_a not in overlapping_boxes:
                new_boxes.add(tuple(box_a))
        boxes = list(new_boxes)
        # Stop trying to merge if we didn't find any
        if not overlapping_boxes:
            break
    return boxes

# Find bounding boxes around contours made of the given colors
# Optional: Filter by size, draw on a debug image, merge overlapping boxes, remove nested boxes
def find_boxes(image_np, target_colors=[], min_size=0, remove_nested=False, max_size=float("inf"), draw=None, box_color='magenta', tolerance=2, merge_overlapping=False, width=1):
    # In the image, find pixels close enough to any of the target colors
    mask = None
    for target_color in target_colors:
        target_color = np.array(target_color)
        dist = np.linalg.norm(image_np - target_color, axis=2)
        new_mask = np.array(dist < tolerance).astype(np.uint8) * 255
        if mask is None:
            mask = new_mask
        else:
            mask |= new_mask

    # Create contours around the matching pixels
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Compute bounding boxes for the contours. Filter boxes by size.
    boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if max_size >= w >= min_size and max_size >= h >= min_size:
            boxes.append((x, y, x + w, y + h))
    
    # Remove nested boxes
    if remove_nested:
        boxes = remove_nested_boxes(boxes)

    # Merge overlapping boxes
    if merge_overlapping:
        boxes = merge_overlapping_boxes(boxes)

    # Draw the boxes
    if draw is not None:
        for box in boxes:
            draw.rectangle(box, outline=box_color, width=width)

    return boxes

# Draw boxes on the debug image
def draw_boxes(boxes, box_color, draw, width=1):
    if draw is None:
        return
    for box in boxes:
        draw.rectangle(box, outline=box_color, width=width)

# Convert an image greyscale and normalize to [0, 1]
def normalize_greyscale(img):
    return np.array(img.convert("L")) / 255.0

# Load a mask in normalized greyscale
def load_mask(path):
    return normalize_greyscale(Image.open(path))

# Load multiple masks, in normalized greyscale
def load_masks(paths):
    return [load_mask(path) for path in paths]

# Find distance between the image and the mask (resize the mask)
# TODO: rename
def distance_to_mask(img_np, mask_np):
    # Scale mask to image size
    mask_resized = Image.fromarray((mask_np * 255).astype(np.uint8)).resize(img_np.shape[::-1], Image.Resampling.BILINEAR)
    # Convert to normalized array
    mask_resized_np = np.array(mask_resized) / 255.0
    # Calculate the L2 distance between the mask and image
    return np.linalg.norm(img_np - mask_resized_np)

# Return the label of the mask that matches the image best
# TODO: rename
def classify_symbol(img, masks):
    img_np = normalize_greyscale(img)
    scores = {label: float("inf") for label in masks}
    for label, mask_nps in masks.items():
        for mask_np in mask_nps:
            score = distance_to_mask(img_np, mask_np)
            scores[label] = min(score, scores[label])
    closest = min(scores, key=scores.get)
    return closest, scores

# Parse a screenshot from Tango app (dark mode) into a puzzle
def parse_tango_app_dark_mode(image_path, debug=False):
    # TODO: Used named tuples or something
    # TODO: Reorder values and parameters to find_boxes to make more sense
    # TODO: Rename variable
    # Parameters for find_boxes. For each cell type, what color, outline, min size, merge overlapping, max size, remove contained, tolerance
    target_colors = {
        Cell.BLANK: ([[85, 84, 84]], 'grey', 50, False, 160, 2, True),
        Cell.YELLOW: ([[57, 55, 55], [244, 221, 135], [209, 190, 121]], 'darkorange', 20, True, 170, 2, True),
        Cell.BLUE: ([[170, 176, 212], [195, 204, 250]], 'blue', 20, True, 170, 2, True),
    }

    # What color to look for in each cell, to figure out the type
    # TODO: Rename variable
    colors = {
        Cell.BLANK: [85, 84, 84],
        Cell.YELLOW: [244, 220, 135],
        Cell.BLUE: [195, 204, 250],
    }

    # What color to make the edges in our debug image
    edge_borders = {Edge.SAME: "lightgreen", Edge.DIFFERENT: "red", Edge.BLANK: None}

    # What color to make the cells in our debug image
    cell_borders = {Cell.BLUE: "blue", Cell.YELLOW: "darkorange", Cell.BLANK: None}

    # Load the game image
    image = Image.open(image_path).convert('RGB')
    image_np = np.array(image)

    # Create a copy of the image to draw debug information
    image_draw = draw = None
    if debug:
        image_draw = image.copy()
        draw = ImageDraw.Draw(image_draw)

    border_width = 4

    # Search for the cells
    cell_boxes = []
    for name, (target_colors, box_color, min_size, remove_nested, max_size, tolerance, merge_overlapping) in target_colors.items():
        boxes = find_boxes(image_np, target_colors, min_size, remove_nested, max_size, draw, box_color, tolerance, merge_overlapping, border_width)
        for box in boxes:
            cell_boxes.append(box)

    # Remove nested boxes
    # TODO: Draw boxes after removing the nested ones instead of in for loop above
    cell_boxes = remove_nested_boxes(cell_boxes)
    if debug: print(f"number of cells: {len(cell_boxes)}")

    # Find and draw the board bounding box
    x_min, y_min, x_max, y_max = min([box[0] for box in cell_boxes]), min([box[1] for box in cell_boxes]), max([box[2] for box in cell_boxes]), max([box[3] for box in cell_boxes])
    board_width, board_height = x_max - x_min, y_max - y_min
    if debug: draw.rectangle([x_min, y_min, x_max, y_max], outline='grey', width=border_width)
    if debug: print(f"board box: {(x_min, y_min, x_max, y_max)}")
    if debug: print(f"board size: {board_width} x {board_height}")

    # Calculate the cell width and height
    cell_width = max([box[2] - box[0] for box in cell_boxes])
    cell_height = max([box[3] - box[1] for box in cell_boxes])
    if debug: print(f"cell size: {cell_width:.1f} x {cell_height:.1f}")

    # Find the number of rows and columns
    # Assuming the board is square, the number of cells equals R * C
    num_rows = num_cols = int(math.sqrt(len(cell_boxes)))
    assert len(cell_boxes) == num_rows * num_cols
    if debug: print(f"board dimensions: {num_rows} x {num_cols}")

    # Calculate the gap between cells
    cell_gap = (board_width - (num_cols * cell_width)) / (num_cols - 1)
    if debug: print(f"cell gap: {cell_gap:.1f}")

    # TODO: We don't need both contour finding AND pixel sampling... If we keep both, we should assert they're the same
    # Populate cells by sampling a pixel from each of them
    cells = [[None for _ in range(num_cols)] for _ in range(num_rows)]
    for r in range(num_cols):
        for c in range(num_cols):
            x = int(x_min + (cell_width + cell_gap) * c + 0.83 * cell_width)
            y = int(y_min + (cell_height + cell_gap) * r + 0.83 * cell_height)
            pixel = image_np[y, x, :]
            distances = {name: np.linalg.norm(pixel - color) for name, color in colors.items()}
            closest = min(distances, key=distances.get)
            # TODO: Draw rectangles here instead of in the for loop above?
            cells[r][c] = closest

    # Create a crop of just the board
    board_img_np = image_np[y_min:y_max, x_min:x_max]

    # Search for edges within the board bounds
    edge_boxes = find_boxes(board_img_np, [[239, 221, 212]], 30, True, 70)
    edge_boxes = [[x1 + x_min, y1 + y_min, x2 + x_min, y2 + y_min] for x1, y1, x2, y2 in edge_boxes]
    draw_boxes(edge_boxes, 'cyan', draw, width=border_width)

    # Figure out each edge's type and which row / col it connects
    mask_paths = {
        Edge.DIFFERENT: ["masks/tango-dark/edge-different.png"], 
        Edge.SAME: ["masks/tango-dark/edge-same.png"]
    }
    edge_masks = {label: load_masks(paths) for label, paths in mask_paths.items()}
    edges = defaultdict(dict)
    # Iterate over all the edge boxes we found
    for x1, y1, x2, y2 in edge_boxes:
        # Figure out if the edge is '=' or 'x'
        closest, _ = classify_symbol(image.crop((x1, y1, x2, y2)), edge_masks)
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        # Find the approximate row and column of the edge
        ar = (cy - y_min) / (cell_height + cell_gap)
        ac = (cx - x_min) / (cell_width + cell_gap)
        # If the row is closer to correct than the column, it's right / left
        if abs(int(ar) - ar) < abs(int(ac) - ac):
            left = r, c = int(ar), int(ac)
            right = r, c + 1
            edges[left][right] = edges[right][left] = closest
        # If the column is closer to correct than the row, it's top / bottom
        else:
            top = r, c  = int(ar), int(ac)
            bottom = r + 1, c
            edges[top][bottom] = edges[bottom][top] = closest
        if debug: draw.rectangle((x1, y1, x2, y2), outline=edge_borders[closest], width=border_width)

    # Show debug image
    if debug: image_draw.show()

    # Intialize the puzzle
    puzzle = Puzzle(cells=cells, edges=edges)

    return puzzle

# Parse a screenshot from LinkedIn (light mode) into a puzzle
def parse_linkedin_light_mode(image_path, debug=False):
    # TODO: Make debug line width thicker for larger images

    # Parameters for find_boxes, for starter cells
    # TODO: Rename
    target_colors = {
        Cell.BLANK: ([[237, 235, 231]], 'lightgrey', 30, True, 220, 6, True),
    }

    # Colors used to classify cells
    cell_colors = {
        Cell.BLANK: [255, 255, 255],
        Cell.YELLOW: [255, 179, 31],
        Cell.BLUE: [76, 140, 231],
    }

    # What color to make the edges in our debug image
    edge_borders = {Edge.SAME: "lightgreen", Edge.DIFFERENT: "red", Edge.BLANK: None}

    # What color to make the cells in our debug image
    cell_borders = {Cell.BLUE: "blue", Cell.YELLOW: "darkorange", Cell.BLANK: None}

    # Load game image
    image = Image.open(image_path).convert('RGB')
    image_np = np.array(image)

    # Create a copy of the image to draw debug info on
    image_draw = draw = None
    if debug:
        image_draw = image.copy()
        draw = ImageDraw.Draw(image_draw)

    # Find the board bounding box
    board_box = find_boxes(image_np, [[235, 228, 217]], 190, False, 1250, None, None, 15, False)
    x_min, y_min, x_max, y_max = board_box[0]
    if debug: print(f"board box: {board_box[0]}")

    # Find the board dimensions
    board_width, board_height = x_max - x_min, y_max - y_min
    if debug: print(f"board size: {board_width} x {board_height}")

    # Adjust border width based on board resolution
    border_width = max(1, round(board_width / 200))

    # Draw the border
    if debug: draw.rectangle(board_box[0], outline="grey", width=border_width)

    # Create a crop of just the board
    board_img_np = image_np[y_min:y_max, x_min:x_max]

    # Find starter cells. Only search within the board.
    starter_boxes = []
    for name, (target_colors, box_color, min_size, remove_nested, max_size, tolerance, merge_overlapping) in target_colors.items():
        boxes = find_boxes(board_img_np, target_colors, min_size, remove_nested, max_size, None, None, tolerance, merge_overlapping)
        boxes = [(x1 + x_min, y1 + y_min, x2 + x_min, y2 + y_min) for x1, y1, x2, y2 in boxes]
        draw_boxes(boxes, box_color, draw, width=border_width)
        starter_boxes.extend(boxes)

    # Find the width and height of the cells
    cell_width = statistics.mean([box[2] - box[0] for box in starter_boxes])
    cell_height = statistics.mean([box[3] - box[1] for box in starter_boxes])
    if debug: print(f"cell size: {cell_width:.1f} x {cell_height:.1f}")

    # Guess the number of rows and columns
    num_rows, num_cols = int(board_width / cell_width), int(board_height / cell_height)
    if debug: print(f"board dims: {num_rows} x {num_cols}")

    # Calculate the gap between each cell
    cell_gap = (board_width - (cell_width * num_cols)) / (num_cols + 1)
    if debug: print(f"cell gap: {cell_gap:.1f}")

    # Populate cells by sampling a pixel
    # TODO: Turn into a helper function, since this is used for both LinkedIn and the Tango App. If you do, careful with how pixel is calculated
    cells = [[None for _ in range(num_cols)] for _ in range(num_rows)]
    for r in range(num_rows):
        for c in range(num_cols):
            # Sample a pixel near the middle of the cell
            x = int(x_min + cell_gap + (cell_width + cell_gap) * c + 0.58 * cell_width)
            y = int(y_min + cell_gap + (cell_height + cell_gap) * r + 0.58 * cell_height)
            pixel = image_np[y, x, :]
            # Find which cell color this pixel is closest to
            distances = {name: np.linalg.norm(pixel - color) for name, color in cell_colors.items()}
            closest = min(distances, key=distances.get)
            # Draw the cell and type on the debug image
            if cell_borders[closest] is not None:
                x1 = x_min + cell_gap + (cell_width + cell_gap) * c
                y1 = y_min + cell_gap + (cell_height + cell_gap) * r
                x2, y2 = x1 + cell_width, y1 + cell_height
                rect = list(map(int, [x1, y1, x2, y2]))
                if debug: draw.rectangle(rect, outline=cell_borders[closest], width=border_width)
            cells[r][c] = closest

    # Calculate the size of the edge symbols (e.g. '=', 'x')
    edge_width = edge_height = cell_width / 193 * 54
    if debug: print(f"edge size: {edge_width:.1f} x {edge_height:.1f}")

    # Open the '=' and 'x' edge masks
    edges = defaultdict(dict)
    mask_paths = {
        Edge.DIFFERENT: ["masks/linkedin-light/edge-different.png"],
        Edge.SAME: ["masks/linkedin-light/edge-same.png"],
    }
    edge_masks = {label: load_masks(paths) for label, paths in mask_paths.items()}

    # TODO: Improve edge symbol detection for blurry games (e.g. small, xs)

    # Find edges row-wise by checking every edge for the closest mask
    # Use vertical edge masks for finding blanks
    vertical_edge_masks = [f"masks/linkedin-light/edge-vertical-{first}-{second}.png" for second in ["light", "dark"] for first in ["light", "dark"]]
    edge_masks[Edge.BLANK] = load_masks(vertical_edge_masks)
    for r in range(num_rows):
        for c in range(num_cols - 1):
            # Find the location of the edge we want to check
            x1 = x_min + cell_gap + (cell_width + cell_gap) * c + cell_width + cell_gap * 0.5 - 0.5 * edge_width
            y1 = y_min + cell_gap + (cell_height + cell_gap) * r + cell_height * 0.5 - 0.5 * edge_height
            x2, y2 = x1 + edge_width, y1 + edge_height
            rect = list(map(int, [x1, y1, x2, y2]))
            # Check if there is an edge and what type if so
            closest, _ = classify_symbol(image.crop(rect), edge_masks)
            # Draw the edge type
            if edge_borders[closest] is not None:
                if debug: draw.rectangle(rect, outline=edge_borders[closest], width=border_width)
            left, right = (r, c), (r, c + 1)
            edges[left][right] = edges[right][left] = closest

    # Find edges column-wise by checking every edge for the closest mask
    # Use horizontal edge masks for finding blanks
    horizontal_edge_masks = [f"masks/linkedin-light/edge-horizontal-{first}-{second}.png" for second in ["light", "dark"] for first in ["light", "dark"]]
    edge_masks[Edge.BLANK] = load_masks(horizontal_edge_masks)
    for r in range(num_rows - 1):
        for c in range(num_cols):
            # Find the location of the edge we want to check
            x1 = x_min + cell_gap + (cell_width + cell_gap) * c + cell_width * 0.5 - 0.5 * edge_width
            y1 = y_min + cell_gap + (cell_height + cell_gap) * r + cell_height + cell_gap * 0.5 - 0.5 * edge_height
            x2, y2 = x1 + edge_width, y1 + edge_height
            rect = list(map(int, [x1, y1, x2, y2]))
            # Check if there is an edge and what type if so
            closest, _ = classify_symbol(image.crop(rect), edge_masks) 
            # Draw the edge type
            if edge_borders[closest] is not None:
                if debug: draw.rectangle(rect, outline=edge_borders[closest], width=border_width)
            top, bottom = (r, c), (r + 1, c)
            edges[top][bottom] = edges[bottom][top] = closest

    # Show debug image
    if debug: image_draw.show()

    # Initialize puzzle
    puzzle = Puzzle(cells=cells, edges=edges)
    
    return puzzle

### Test text input

In [17]:
# Parse puzzle from text file (see examples)

# text_file = 'text_inputs/tango/genius-1.txt'
# text_file = 'text_inputs/tango/genius-2.txt'
# text_file = 'text_inputs/tango/expert-10.txt'
# text_file = 'text_inputs/tango/beginner-2.txt'
text_file = 'text_inputs/linkedin/210.txt'


start = time.time()
puzzle = parse_text_file(text_file)
print(f"Parsed text in {time.time() - start:0.3f} seconds")

start = time.time()
soln = solve(deepcopy(puzzle))
print(f"Solved in {time.time() - start:0.3f} seconds")

puzzle.print()
soln.print()

Parsed text in 0.002 seconds
Solved in 0.002 seconds

      0|0   1|1   0|2   1|1   1|1   0|0
                                             
0|0             =           =             0|0
       =                             x       
2|0          ☾                 ☾          2|0
                                             
1|1                ☼     ☾                1|1
       x                             =       
0|2                ☼     ☼                0|2
                                             
0|2          ☼                 ☼          0|2
       =                             =       
0|0             =           =             0|0
                                             
      0|0   1|1   0|2   1|1   1|1   0|0


      3|3   3|3   3|3   3|3   3|3   3|3
                                             
3|3    ☼  x  ☾  =  ☾  x  ☼  =  ☼  x  ☾    3|3
       =                             x       
3|3    ☼     ☾     ☾     ☼     ☾     ☼    3|3
       x                             x    

### Test Tango App image input

In [18]:
# Parse puzzle from Tango App screenshot (dark mode)

# img_path = 'image_inputs/tango-dark/genius-2.png'
# img_path = 'image_inputs/tango-dark/beginner-2.png'
# img_path = 'image_inputs/tango-dark/genius-10.png'
img_path = 'screenshots/tango-genius-2-input.png'

start = time.time()
puzzle = parse_tango_app_dark_mode(img_path)
print(f"Parsed image in {time.time() - start:0.3f} seconds")

start = time.time()
soln = solve(deepcopy(puzzle))
print(f"Solved in {time.time() - start:0.3f} seconds")

puzzle.print()
soln.print()

Parsed image in 0.588 seconds
Solved in 0.073 seconds

      3|2   3|3   1|3   1|2   1|1   1|2   3|0   2|1   0|3   1|1   2|1   2|2   2|4   1|4
                                                                                             
3|2          ☾     ☾                             ☼                             ☾     ☼    3|2
                                                 x                       =                   
4|4    ☾           ☼           ☾     ☼                             ☾     ☾     ☼     ☼    4|4
                                                                                             
0|1                   x                                               x        ☼          0|1
                               =                       =                 =                   
2|2                                  ☾     ☾                       ☼  =  ☼                2|2
                                                                         x                   
2|1    ☾   

### Test LinkedIn image input

In [19]:
# Parse puzzle from LinkedIn screenshot (light mode)

# img_path = 'image_inputs/linkedin-light/210-pc-large.png'
# img_path = 'image_inputs/linkedin-light/210-pc-medium.png'
# img_path = 'image_inputs/linkedin-light/210-pc-small.png'
# img_path = 'image_inputs/linkedin-light/210-pc-xs.png'
# img_path = 'image_inputs/linkedin-light/210.png'
# img_path = 'image_inputs/linkedin-light/209.png'
# img_path = 'image_inputs/linkedin-light/208.png'
img_path = 'screenshots/linkedin-210-input.png'

start = time.time()
puzzle = parse_linkedin_light_mode(img_path, debug=False)
print(f"Parsed image in {time.time() - start:0.3f} seconds")

start = time.time()
soln = solve(deepcopy(puzzle))
print(f"Solved in {time.time() - start:0.3f} seconds")

puzzle.print()
soln.print()

Parsed image in 0.277 seconds
Solved in 0.004 seconds

      0|0   1|1   0|2   1|1   1|1   0|0
                                             
0|0             =           =             0|0
       =                             x       
2|0          ☾                 ☾          2|0
                                             
1|1                ☼     ☾                1|1
       x                             =       
0|2                ☼     ☼                0|2
                                             
0|2          ☼                 ☼          0|2
       =                             =       
0|0             =           =             0|0
                                             
      0|0   1|1   0|2   1|1   1|1   0|0


      3|3   3|3   3|3   3|3   3|3   3|3
                                             
3|3    ☼  x  ☾  =  ☾  x  ☼  =  ☼  x  ☾    3|3
       =                             x       
3|3    ☼     ☾     ☾     ☼     ☾     ☼    3|3
       x                             x   

### Test that text v.s. image parsing

In [20]:
# Test that parsing from text is the same result as parsing from image

start = time.time()
puzzle_from_text = parse_text_file('text_inputs/tango/genius-2.txt') 
print(f"Parsed text in {time.time() - start:0.3f} seconds")

start = time.time()
puzzle_from_img = parse_tango_app_dark_mode('image_inputs/tango-dark/genius-2.png')
print(f"Parsed image in {time.time() - start:0.3f} seconds")

assert puzzle_from_text == puzzle_from_img

Parsed text in 0.001 seconds
Parsed image in 1.227 seconds


### TODOs

In [21]:
# TODO: Organize into functions / classes / files
# TODO: Create a fuction that can dump the puzzle as an input.txt file
# TODO: Dedupe code between LinkedIn and Tango App
# TODO: Make image parsing faster